<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/simulacao_carro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

In [ ]:
import simpy
import numpy.random as rd

class Car:

  def __init__(self,env,agente,car_number,time_matrix):
    self.env = env
    self.agente = agente              ##Referência para o agente que controla os carros
    self.number = car_number          ##Número único do carro
    self.time_matrix = time_matrix    ##Matriz de tempos médios entre locais
    self.location = 0                 ##Variável que guarda o local do veículo. É uma variável de estado. Inicia em zero.
    self.busy = False                   ##Variável de estado. True = carro está realizando uma corrida
    self.life_proc = env.process(self.life())     ##Processo de vida do veículo
    self.start_driving = env.event()              ##O carro espera esse evento ocorrer para iniciar a corrida

  def life(self):

    """
    Processo que define a vida do veículo.
    """
    while True:
        ##Inicia esperando o sinal do agente para começar uma corrida (evento start_driving)
        received_call = yield self.start_driving
        self.busy = True
        atraso = yield self.env.process(self.ride(received_call))  ##Processo de viagem
        self.busy = False
        self.agente.free_car.succeed((self.number,atraso))         ##Dispara o evento free_car (Avisa ao agente que está livre e retorna o atraso da viagem)
        self.start_driving = self.env.event()                      ##Recria o evento (fica em espera novamente)

  def ride(self,call):
    """
    Processo que representa a realização de uma viagem.
    call: chamada repassada pelo agente
    """
    print("Carro",self.number,": iniciando viagem no local", self.location, "em t = ",self.env.now)
    print("Carro",self.number,": atendendo call n.", call.serial_number, "com origem",call.origem, "e destino",call.dest)
    duracao_viagem_origem = self.time_matrix[self.location,call.origem]
    yield self.env.timeout(duracao_viagem_origem)   ##Tempo realizando a viagem até a origem da chamada.
    print("Carro",self.number,": chegou no local da chamada",call.serial_number,"em t = ",self.env.now)
    self.location = call.origem
    atraso = self.env.now - call.birth_time          ##Computa o atraso (tempo entre o nascimento da chamada até o veículo chegar até ela)
    print("Carro", self.number,": atraso = ", atraso)
    duracao_viagem_destino = self.time_matrix[self.location,call.dest]
    yield self.env.timeout(duracao_viagem_destino)   ##Tempo realizando a viagem até o destino da chamada
    print("Carro",self.number,": chegou no destino da chamada",call.serial_number,"em t = ",self.env.now)
    self.location = call.dest
    print("Carro", self.number,": parando de dirigir em t = ", self.env.now)
    
    return atraso     ##Retorna o atraso no atendimento à chamada

class Agent:

    """
    O agente controla um conjunto de carros registrado na lista self.cars.
    A lista self.lista_calls contém as chamadas em espera.
    """

    def __init__(self,env):
        self.env = env
        self.cars = []                  ##Lista com os carros controlados
        self.lista_calls = []           ##Lista que guarda as calls
        self.atraso_total = 0           ##Variável que acumula o atraso dos veículos
        self.free_car = env.event()     ##Esse evento é disparado por um carro quando fica livre
        self.new_call = env.event()     ##Esse evento é disparado quando a lista de calls está vazia e chega uma nova call
        self.controller_proc = env.process(self.control())

    def control(self):

        """
        Processo que define as ações de controle do agente.
        """

        while True:
            yield self.free_car | self.new_call         ##Espera os eventos free_car ou new_call

            ##Verifica qual evento ocorreu
            if self.new_call.triggered:
                print("Agente: nova call em t = ",self.env.now)
                self.new_call = self.env.event()      ##Restabelece o evento

                ##Testa se há algum veículo livre
                veiculo_livre = False
                free_cars = []
                for car in self.cars:
                  if car.busy == False:
                    free_cars.append(car.number)    ##Guarda o número do carro que está livre
                    veiculo_livre = True

                print("Agente: veículos livres = ",free_cars)

                if veiculo_livre == True:                ##Note que se for falso, ignora o evento
                    selected_call = self.lista_calls.pop()       ##Retira a call da lista (deve ser única)
                    ##Escolhe aleatoriamente um carro da lista de carros livres:
                    car_index = rd.randint(0,len(free_cars))
                    selected_car_number = free_cars[car_index]
                    selected_car = self.cars[selected_car_number]
                    print("Agente: viagem alocada ao carro", selected_car.number, "em t =",self.env.now)
                    selected_car.start_driving.succeed(selected_call)
                    #selected_car.start_driving = self.env.event()    ##Recria o evento do carro

            if self.free_car.triggered:
                car_number, atraso = self.free_car.value  ##Verifica qual carro ficou livre e recupera o valor do atraso
                self.atraso_total+=atraso    ##Incrementa o atraso
                print("Agente: carro",car_number," ficou livre em t = ", self.env.now)
                self.free_car = self.env.event()       ##Restabelece o evento
                if len(self.lista_calls) > 0:
                    print("Agente: lista de calls tem",len(self.lista_calls), "calls")
                    selected_call = self.lista_calls.pop(0)       ##Retira a call no início da lista. Aqui é que é o cérebro do agente.
                    print("Agente: retira uma call da lista e aloca um carro")
                    self.cars[car_number].start_driving.succeed(selected_call)    ##Manda carro iniciar viagem
                else:
                  print("Agente: lista de chamadas está vazia")

class Call:
  """
  Uma chamada para realizar uma viagem.
  """

  def __init__(self, serial_number, birth_time, origem, dest):
    self.serial_number = serial_number
    self.birth_time = birth_time
    self.origem = origem
    self.dest = dest

def arrival_process(env, tec,locations,agente):

  """
  Processo de chegada de calls, seguindo um processo de Poisson.
  """

  serial_numbers = 0
  while True:
     possible_locations = list(locations)
     origem  = rd.choice(possible_locations)
     possible_locations.remove(origem)
     destino = rd.choice(possible_locations)
     call = Call(serial_numbers,env.now,origem,destino)

     if len(agente.lista_calls) == 0:
         agente.new_call.succeed()

     agente.lista_calls.append(call)
     interarrival_time = rd.exponential(tec)
     serial_numbers+=1
     yield env.timeout(interarrival_time)

def simulation(n_cars,locations,tec,time_matrix,sim_length,seed):

    rd.seed(seed)
    
    ##Definição do modelo
    env = simpy.Environment()    ##Cria o ambiente
    agente = Agent(env)          ##Cria o agente inteligente (controller)
    cars = [Car(env,agente,i,time_matrix) for i in range(n_cars)]    ##Cria a lista de carros disponíveis   
    agente.cars = cars    ##Registra os carros no agente
    arrival_process_1 = env.process(arrival_process(env,tec,locations,agente))    ##Cria processo de chegada de chamadas (calls)
    env.run(until=sim_length)

    return agente.atraso_total

##PARAMETROS
n_cars = 3
n_locations = 10
tec = 10
sim_length = 1000
locations = list(range(0,n_locations))
time_matrix = rd.randint(10, 100, size=(n_locations,n_locations))

atraso_total = simulation(n_cars,locations,tec,time_matrix,sim_length,39)
print("ATRASO TOTAL = ", atraso_total)
print("ATRASO MEDIO = ", atraso_total/sim_length)



Agente: nova call em t =  0
Agente: veículos livres =  [0, 1, 2]
Agente: viagem alocada ao carro 0 em t = 0
Carro 0 : iniciando viagem no local 0 em t =  0
Carro 0 : atendendo call n. 0 com origem 9 e destino 0
Agente: nova call em t =  17.170335903090265
Agente: veículos livres =  [1, 2]
Agente: viagem alocada ao carro 1 em t = 17.170335903090265
Carro 1 : iniciando viagem no local 0 em t =  17.170335903090265
Carro 1 : atendendo call n. 1 com origem 4 e destino 9
Agente: nova call em t =  43.5795532746057
Agente: veículos livres =  [2]
Agente: viagem alocada ao carro 2 em t = 43.5795532746057
Carro 2 : iniciando viagem no local 0 em t =  43.5795532746057
Carro 2 : atendendo call n. 2 com origem 9 e destino 0
Carro 0 : chegou no local da chamada 0 em t =  54
Carro 0 : atraso =  54
Agente: nova call em t =  68.78511647076769
Agente: veículos livres =  []
Carro 1 : chegou no local da chamada 1 em t =  94.17033590309026
Carro 1 : atraso =  77.0
Carro 2 : chegou no local da chamada 2 em t